In [1]:
import nextroute
import nextmv


import os
import json
import folium

/Users/mac-Z03AMITR/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#Vehicle Capacity and Locations
vehicle_capacity = {
    "bunnies": 20,
    "rabbits": 10
}

vehicle_start_end_location = {
    "start_location": {
        "lat": 35.791729813680874,
        "lon": -78.7401685145487
    },
    "end_location": {
        "lat": 35.791729813680874,
        "lon": -78.7401685145487
    }
}

vehicle_speed = 34.44

#Stop Information
stop_default = {
    # "duration": 300,
    # "quantity": {
    #     "bunnies": -1,
    #     "rabbits": -1
    # },
    # "unplanned_penalty": 200000,
    # "target_arrival_time": "2023-01-01T10:00:00Z",
    "early_arrival_time_penalty": 1.5,
    "late_arrival_time_penalty": 1.5
}

# List of all stops
stops_data = [
    {"id": "P001", "location": {"lon": -53.987533, "lat": 47.291122}, "demand": 0, "duration": 18000, "quantity": 0},
    {"id": "F001", "location": {"lon": -9.27148, "lat": 41.9296}, "demand": 300, "duration": 43200.0, "quantity": 0, "succeeds": "P001",  "unplanned_penalty": 500000},
    {"id": "M001", "location": {"lon": -62.503349, "lat": 44.518828}, "demand": 500, "duration": 70200.0, "quantity": 0, "succeeds": "F001",  "unplanned_penalty": 500000},
    {"id": "OF001", "location": {"lon": -75.59966, "lat": 34.937392}, "demand": 180, "duration": 0, "quantity": 0, "succeeds": "F001",  "unplanned_penalty": 500000, "target_arrival_time": "2025-03-25T00:00:00-00:00"},
]

# Vehicle Information
vehicles_data = [
   {'id': 'ATLAS', 'start_location': {'lon': -19.8178032, 'lat': 34.937392},'end_location': {'lon': -75.59966, 'lat': 34.937392}, 'start_time': '2025-02-25T00:00:00-00:00', 'end_time': '2025-03-25T00:00:00-00:00', 'capacity': 5},
   {'id': 'BOKALIFT', 'start_location': {'lon': -62.5415968, 'lat': 62.1273427},'end_location': {'lon': -75.59966, 'lat': 34.937392} ,'start_time': '2025-03-01T00:00:00-00:00', 'end_time': '2025-03-25T00:00:00-00:00', 'capacity': 1} 
]

# final structure
final_combined_data = {
    "defaults": {
        "vehicles": {
            # "capacity": vehicle_capacity,
            # "start_end_location": vehicle_start_end_location,
            "speed": vehicle_speed
        },
        #"stops": stop_default
    },
    "stops": stops_data,
    "vehicles": vehicles_data
}
final_combined_data

{'defaults': {'vehicles': {'speed': 34.44}},
 'stops': [{'id': 'P001',
   'location': {'lon': -53.987533, 'lat': 47.291122},
   'demand': 0,
   'duration': 18000,
   'quantity': 0},
  {'id': 'F001',
   'location': {'lon': -9.27148, 'lat': 41.9296},
   'demand': 300,
   'duration': 43200.0,
   'quantity': 0,
   'succeeds': 'P001',
   'unplanned_penalty': 500000},
  {'id': 'M001',
   'location': {'lon': -62.503349, 'lat': 44.518828},
   'demand': 500,
   'duration': 70200.0,
   'quantity': 0,
   'succeeds': 'F001',
   'unplanned_penalty': 500000},
  {'id': 'OF001',
   'location': {'lon': -75.59966, 'lat': 34.937392},
   'demand': 180,
   'duration': 0,
   'quantity': 0,
   'succeeds': 'F001',
   'unplanned_penalty': 500000,
   'target_arrival_time': '2025-03-25T00:00:00-00:00'}],
 'vehicles': [{'id': 'ATLAS',
   'start_location': {'lon': -19.8178032, 'lat': 34.937392},
   'end_location': {'lon': -75.59966, 'lat': 34.937392},
   'start_time': '2025-02-25T00:00:00-00:00',
   'end_time': '2

In [3]:
class DecisionModel(nextmv.Model):
    def solve(self, input: nextmv.Input) -> nextmv.Output:
        """Solves the given problem and returns the solution."""

        nextroute_input = nextroute.schema.Input.from_dict(input.data)
        nextroute_options = nextroute.Options.extract_from_dict(input.options.to_dict())
        nextroute_output = nextroute.solve(nextroute_input, nextroute_options)

        return nextmv.Output(
            options=input.options,
            solution=nextroute_output.solutions[0].to_dict(),
            statistics=nextroute_output.statistics.to_dict(),
        )

In [4]:
parameters = []

default_options = nextroute.Options()
for name, default_value in default_options.to_dict().items():
    parameters.append(nextmv.Parameter(name.lower(), type(default_value), default_value, name, False))

options = nextmv.Options(*parameters)

In [5]:
model = DecisionModel()

In [6]:
input = nextmv.Input(data=final_combined_data, options=options)
output = model.solve(input)
print(json.dumps(output.solution, indent=2))

{
  "unplanned": [],
  "vehicles": [
    {
      "id": "ATLAS",
      "route": [
        {
          "stop": {
            "id": "ATLAS-start",
            "location": {
              "lat": 34.937392,
              "lon": -19.8178032
            }
          },
          "arrival_time": "2025-02-25T00:00:00Z",
          "cumulative_travel_duration": 0.0,
          "end_time": "2025-02-25T00:00:00Z",
          "start_time": "2025-02-25T00:00:00Z",
          "travel_duration": 0.0
        },
        {
          "stop": {
            "id": "P001",
            "location": {
              "lat": 47.291122,
              "lon": -53.987533
            }
          },
          "arrival_time": "2025-02-26T01:20:21Z",
          "cumulative_travel_distance": 3141652.0,
          "cumulative_travel_duration": 91221.0,
          "duration": 18000.0,
          "end_time": "2025-02-26T06:20:21Z",
          "start_time": "2025-02-26T01:20:21Z",
          "travel_distance": 3141652.0,
          "travel

In [ ]:
m = folium.Map(location=[35.398699, -42.317552], zoom_start=3)

data = final_combined_data["stops"]
for item in data:
    folium.Marker(
        location=[item['location']['lat'], item['location']['lon']],
        popup=f"lat: {item['location']['lat']}, lon: {item['location']['lon']}",  # Corrected popup format
        tooltip=item['id']
    ).add_to(m)


    
for item in final_combined_data["vehicles"]:     
    folium.Marker(
            location=[item['start_location']['lat'], item['start_location']['lon']],
            popup="Vehicle start End location",
            icon=folium.Icon(color="green"),
        ).add_to(m)

In [8]:
m

In [9]:
# trail_coordinates = []
# for values in output.solution['unplanned']:
#     trail_coordinates.append(
#         (values['location']['lat'], values['location']['lon'])
#     )
    
# trail_coordinates

# folium.PolyLine(trail_coordinates, tooltip="Coast").add_to(m)

# m

In [10]:
output.solution.keys()

dict_keys(['unplanned', 'vehicles', 'objective'])

In [11]:
output.solution["objective"]

{'name': '1 * vehicles_duration + 1 * unplanned_penalty',
 'objectives': [{'name': 'vehicles_duration',
   'factor': 1,
   'base': 585173.8695342541,
   'value': 585173.8695342541},
  {'name': 'unplanned_penalty', 'factor': 1, 'value': 0}],
 'value': 585173.8695342541}

In [12]:
output.solution['vehicles']

[{'id': 'ATLAS',
  'route': [{'stop': {'id': 'ATLAS-start',
     'location': {'lat': 34.937392, 'lon': -19.8178032}},
    'arrival_time': '2025-02-25T00:00:00Z',
    'cumulative_travel_duration': 0.0,
    'end_time': '2025-02-25T00:00:00Z',
    'start_time': '2025-02-25T00:00:00Z',
    'travel_duration': 0.0},
   {'stop': {'id': 'P001', 'location': {'lat': 47.291122, 'lon': -53.987533}},
    'arrival_time': '2025-02-26T01:20:21Z',
    'cumulative_travel_distance': 3141652.0,
    'cumulative_travel_duration': 91221.0,
    'duration': 18000.0,
    'end_time': '2025-02-26T06:20:21Z',
    'start_time': '2025-02-26T01:20:21Z',
    'travel_distance': 3141652.0,
    'travel_duration': 91221.0},
   {'stop': {'id': 'F001', 'location': {'lat': 41.9296, 'lon': -9.27148}},
    'arrival_time': '2025-02-27T10:53:18Z',
    'cumulative_travel_distance': 6681293.0,
    'cumulative_travel_duration': 193998.0,
    'duration': 43200.0,
    'end_time': '2025-02-27T22:53:18Z',
    'start_time': '2025-02-27T

In [13]:
vehicle0_sol = output.solution['vehicles'][0]
vehicle1_sol = output.solution['vehicles'][1]

In [14]:
trail_coordinates_for_vehcile0 = []
trail_coordinates_for_vehcile1 = []


for route in vehicle0_sol['route']:
    stop = route['stop']
    trail_coordinates_for_vehcile0.append((stop['location']['lat'], stop['location']['lon']))
    
for route in vehicle1_sol['route']:
    stop = route['stop']
    trail_coordinates_for_vehcile1.append((stop['location']['lat'], stop['location']['lon']))

In [15]:
folium.PolyLine(
    locations=trail_coordinates_for_vehcile0,
    color="#FF0000",
    weight=5,
    tooltip="Vehicle 0 Stops"
).add_to(m)


folium.PolyLine(
    locations=trail_coordinates_for_vehcile1,
    color="#0000FF",
    weight=5,
    tooltip="Vehicle 1 Stops"
).add_to(m)

m